# 4. 画像データを取得し、ローカルに保存する
このケースでは、以下の方法を中心に解説します。

* サーバー側に保存されたM-JPEGデータをバイナリデータとして取得する
* 画像ファイル(.jpg)として保存する

## シナリオ
本シナリオでは、iOSアプリケーション  **Visual M2M Motion** を活用し、ストリーミングされた動画データを画像として保存します。

## 事前準備
本シナリオを実施する前に、以下を用意する必要があります。

- 計測用のエッジ
- Visual M2M Motion アプリでアップロードした計測(画像を含む)


### 使用データ
本シナリオでは、事前に以下のデータをサーバー側に準備する必要があります。  

|データ項目|本シナリオで登場するデータ名|
|:---|:---|
|時系列データを登録するエッジ|sdk_edge1|
|計測情報 (※)|measurement1|

(※) 使用する時系列データは **「1. 取得した時系列データをCSVで保存する」**  と同様に、Visual M2M Motionを使用して時系列データを登録します。

### 計測データの作成と時系列データのアップロード
**Visual M2M Motion** を使用してデータをアップロードします。 計測を登録後、 **Visual M2M Data Visualizer** にて計測が新しく生成されたことを確認してください。

### (補足) Visual M2M Motion の操作手順
1. iPhoneでVisual M2M Motionを起動し、 `sdk_edge1` としてサインインする。
2. Settings > Video にて、以下の設定を行う。
    1. Streaming を有効にする。
    2. Save to server を有効にする。
    3. Codec をJPEGに設定する。
3. Main画面に戻り、画面中央の▶をタップする。
4. 計測が開始した画面を確認する。
5. ■をタップし、計測を終了する。

VM2M Data Viz の 「Stored Data」に計測が表示されていれば完了です。

<img src="https://github.com/aptpod/intdash-py-sample-codes/blob/master/img/img1.png?raw=true">

### パッケージのimportとクライアントの生成
`intdash.Client` に与える `url` は intdashサーバーの環境情報から、 `edge_token` はログイン用エッジで発行したトークンを指定してください。  
(※ `username` と `password` でのログインも可能ですが、継続して動作する場合はエッジトークンの使用を推奨します)

In [8]:
import pandas as pd

import intdash
from intdash import timeutils

# Create client
client = intdash.Client(
    url = "https://example.intdash.jp",
    edge_token = "your_token",
)

これで、事前準備は終了です。

## 計測に使用したエッジを取得する

In [9]:
edges = client.edges.list(name='sdk_edge1')
sdk_edge1 = edges[0]

In [10]:
sdk_edge1.name

'sdk_edge1'

## 時系列データを取得する
本シナリオでは、時系列データの取得には `client.data_points` エンドポイントを使用します。  信号定義からデータを取得する場合は ``labels`` を指定しますが、本ケースでは信号定義を使用しないため、 `id_queries` で取得します。  
`start` および `end` は前述の手順で作成した計測を含む時刻に変更してください。

※通常は ``data_id`` を指定しますが、JPEGデータの場合は ``data_id`` が固定されているため指定する必要はありません。詳細は、ドキュメント記載の [intdash.data.Jpeg](https://docs.intdash.jp/sdk/python/latest/ja/data.html#intdash.intdash.data.JPEG)を確認してください。

In [11]:
dps = client.data_points.list(
    edge_name='sdk_edge1',
    start=timeutils.str2timestamp('2020-07-17 00:00:00+09:00'), # change appropriately.
    end=timeutils.str2timestamp('2020-07-18 00:00:00+09:00'), # change appropriately.
    id_queries=[
        intdash.IdQuery(
            data_type=intdash.DataType.jpeg,
            channel=1
        )
    ]
)

## 取得したデータを画像ファイルとして保存する
取得したバイナリデータを、そのままJPEGとして保存します。
ここではサンプルとして、取得した時系列データのうち、最初のデータポイントのみを保存します。

In [20]:
data = dps[0]  #  Specifies first data.

# Define the file name as a timestamp.
with open(f'./{pd.Timestamp(data.time).value}.jpg', 'wb') as fout:
    fout.write(data.data_payload)

全時系列データを保存したい場合は、上記を繰り返します。

In [22]:
import os 

save_dir='./images'

for d in dps:
    
    if not os.path.isdir(save_dir):
            os.makedirs(save_dir)
            
    # Define the file name as a timestamp.
    with open(f'{save_dir}/{pd.Timestamp(d.time).value}.jpg', 'wb') as fout:
        fout.write(d.data_payload)